In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
import xarray as xr
import tensorflow as tf
from shared.models import *
from shared.utilities import *
from shared.training import train_and_evaluate, split_data_on_participants, k_fold_cross_validate
from shared.normalization import *
from shared.generators import *
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async

2023-09-28 09:41:06.423923: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 09:41:06.939056: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


### Set up data

In [2]:
data_path = Path("data/sat1/split_stage_data.nc")

data = xr.load_dataset(data_path)

In [2]:
data_path = Path("data/sat1/split_stage_data_unprocessed_500hz.nc")

data = xr.load_dataset(data_path)

In [3]:
train_data, val_data, test_data = split_data_on_participants(data, 60, norm_dummy)

In [4]:
model = SAT1Deep(len(data.channels), len(data.samples), len(data.labels))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

2023-09-25 14:13:55.562065: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-25 14:13:55.676804: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-25 14:13:55.676875: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-25 14:13:55.684773: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-25 14:13:55.684828: I tensorflow/compile

In [20]:
model = SAT1TopologicalConv(5, 8, len(data.samples), len(data.labels))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [4]:
model = SAT1Base(len(data.channels), len(data.samples), len(data.labels))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

2023-09-28 09:41:14.302183: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-28 09:41:14.399367: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-28 09:41:14.399447: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-28 09:41:14.401945: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-28 09:41:14.402003: I tensorflow/compile

In [6]:
train_and_evaluate(
    model,
    train_data,
    val_data,
    test_data,
    epochs=20,
    logs_path=Path("logs/")
)

Epoch 1/20


2023-09-28 09:41:33.699547: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-09-28 09:41:35.126021: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-28 09:41:35.139844: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x562506be52d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-28 09:41:35.139874: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-09-28 09:41:35.162536: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-28 09:41:35.329644: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

765/765 [==============================] - 16s 14ms/step - loss: 0.8571 - accuracy: 0.6727 - val_loss: 0.6429 - val_accuracy: 0.7602
Epoch 2/20
765/765 [==============================] - 11s 14ms/step - loss: 0.6112 - accuracy: 0.7775 - val_loss: 0.5544 - val_accuracy: 0.7876
Epoch 3/20
765/765 [==============================] - 11s 14ms/step - loss: 0.5227 - accuracy: 0.8045 - val_loss: 0.5337 - val_accuracy: 0.8065
Epoch 4/20
765/765 [==============================] - 11s 14ms/step - loss: 0.4825 - accuracy: 0.8220 - val_loss: 0.5014 - val_accuracy: 0.8083
Epoch 5/20
765/765 [==============================] - 11s 14ms/step - loss: 0.4389 - accuracy: 0.8392 - val_loss: 0.5255 - val_accuracy: 0.8123
Epoch 6/20
765/765 [==============================] - 11s 14ms/step - loss: 0.4050 - accuracy: 0.8537 - val_loss: 0.4854 - val_accuracy: 0.8210
Epoch 7/20
765/765 [==============================] - 11s 14ms/step - loss: 0.3655 - accuracy: 0.8658 - val_loss: 0.4938 - val_accuracy: 0.8220
Epo

(<keras.src.callbacks.History at 0x7f306fb7f550>, None)

In [21]:
train_and_evaluate(
    model,
    train_data,
    val_data,
    test_data,
    epochs=20,
    logs_path=Path("logs/"),
    gen_kwargs={"shape_topological": True},
)

Epoch 1/20
765/765 [==============================] - 12s 14ms/step - loss: 0.7901 - accuracy: 0.6996 - val_loss: 0.6132 - val_accuracy: 0.7712
Epoch 2/20
765/765 [==============================] - 10s 13ms/step - loss: 0.5591 - accuracy: 0.7946 - val_loss: 0.5193 - val_accuracy: 0.8043
Epoch 3/20
765/765 [==============================] - 10s 13ms/step - loss: 0.4688 - accuracy: 0.8308 - val_loss: 0.5023 - val_accuracy: 0.8147
Epoch 4/20
765/765 [==============================] - 10s 13ms/step - loss: 0.4113 - accuracy: 0.8460 - val_loss: 0.4798 - val_accuracy: 0.8274
Epoch 5/20
765/765 [==============================] - 10s 13ms/step - loss: 0.3623 - accuracy: 0.8669 - val_loss: 0.4723 - val_accuracy: 0.8294
Epoch 6/20
765/765 [==============================] - 11s 14ms/step - loss: 0.3210 - accuracy: 0.8834 - val_loss: 0.4800 - val_accuracy: 0.8329
Epoch 7/20
765/765 [==============================] - 11s 14ms/step - loss: 0.2844 - accuracy: 0.8963 - val_loss: 0.4799 - val_accuracy:

(<keras.src.callbacks.History at 0x7fa497865d50>, None)

: 

In [19]:
# Run before re-training to clear up VRAM
import gc

gc.collect()
tf.keras.backend.clear_session()
del model

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 154, 30, 1)]      0         
                                                                 
 masking (Masking)           (None, 154, 30, 1)        0         
                                                                 
 conv2d (Conv2D)             (None, 150, 30, 64)       384       
                                                                 
 max_pooling2d (MaxPooling2  (None, 75, 30, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 73, 30, 128)       24704     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 30, 128)       0         
 g2D)                                                        